In [27]:
#Aula 14 - DinoGame
#opengameart.org
#fazer próprios sprites: piskelapp.com

import pygame
from pygame.locals import *
from sys import exit
import os
from random import randrange, choice


pygame.init()
pygame.mixer.init()

diretorio_principal = os.getcwd()
diretorio_imagens = os.path.join(diretorio_principal, 'images')
diretorio_sons = os.path.join(diretorio_principal, 'sounds')

#cores:
branco = (255, 255, 255)
preto = (0, 0, 0)
azul = (0, 0, 255)

pygame.init()

largura = 640
altura = 480

tela = pygame.display.set_mode((largura, altura))
pygame.display.set_caption('Dino')

spriteSheet = pygame.image.load(os.path.join(diretorio_imagens, 'dinoSpritesheet.png')).convert_alpha()

som_colisao = pygame.mixer.Sound(os.path.join(diretorio_sons, 'death_sound.wav'))
som_colisao.set_volume(1)

som_pontuacao = pygame.mixer.Sound(os.path.join(diretorio_sons, 'score_sound.wav'))
som_pontuacao.set_volume(1)

colidiu = False

escolha_obstaculo = choice([0,1])

pontos = 0
velocidade_jogo = 10

def exibe_mensagem(mensagem, tamanho, cor, pos_x, pos_y):
    fonte = pygame.font.SysFont('comicsansms', tamanho, True, False)
    mensagem = f'{mensagem}'
    texto_formatado = fonte.render(mensagem, True, cor)
    tela.blit(texto_formatado, (pos_x, pos_y))
    
def reiniciarJogo ():
    global colidiu, pontos, velocidade_jogo, escolha_obstaculo
    pontos = 0
    velocidade_jogo = 10
    colidiu = False
    cacto.rect.x = largura
    dinoVoador.rect.x = largura
    escolha_obstaculo = choice([0,1])
    dino.rect.y = dino.pos_y_inicial
    dino.pulo = False

class Dino(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.som_pulo = pygame.mixer.Sound(os.path.join(diretorio_sons, 'jump_sound.wav'))
        self.som_pulo.set_volume(1)
        self.imagens_dino = []
        for i in range(3):
            img = spriteSheet.subsurface((i*32, 0), (32, 32))
            img = pygame.transform.scale(img, (32*3, 32*3))
            self.imagens_dino.append(img)
        self.index_lista = 0
        self.image = self.imagens_dino[self.index_lista] 
        self.rect = self.image.get_rect()
        self.mask = pygame.mask.from_surface(self.image)
        self.pos_y_inicial = altura - 64 - 96//2
        self.rect.center = (100,altura-64)
        self.pulo = False
    
    def update(self):
        self.index_lista += 0.25
        if self.index_lista > 2:
            self.index_lista = 0      
        self.image = self.imagens_dino[int(self.index_lista)] 
        
        if self.pulo == True:
            self.rect.y -= 20
            if self.rect.y < 200:
                self.pulo = False
        else:
            self.rect.y += 20
            if self.rect.y > self.pos_y_inicial:
                self.rect.y = self.pos_y_inicial
        
    def pular(self):
        self.pulo = True   
        self.som_pulo.play()

class Nuvens(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        
        self.image = spriteSheet.subsurface((7*32, 0), (32, 32))
        self.image = pygame.transform.scale(self.image, (32*3, 32*3))
        self.rect = self.image.get_rect()
        self.rect.y= randrange(50,200,50)
        self.rect.x = largura - randrange(30,300,90)
        
    def update(self):
        self.rect.x -= velocidade_jogo
        if self.rect.topright[0] < 0:
            self.rect.y= randrange(50,200,50)
            self.rect.x = largura
            
class Chao(pygame.sprite.Sprite):
    def __init__(self, posX):
        pygame.sprite.Sprite.__init__(self)
        self.image = spriteSheet.subsurface((6*32, 0), (32, 32))
        self.image = pygame.transform.scale(self.image, (32*2, 32*2))
        self.rect = self.image.get_rect()
        self.rect.y = altura - 64
        self.rect.x = posX*64
        
    def update(self):
        self.rect.x -= 5
        if self.rect.topright[0] < 0:
            self.rect.x = largura
            
class Cacto (pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.image = spriteSheet.subsurface((5*32, 0), (32, 32))
        self.image = pygame.transform.scale(self.image, (32*2, 32*2))
        self.rect = self.image.get_rect()
        self.mask = pygame.mask.from_surface(self.image)
        self.escolha = escolha_obstaculo
        self.rect.center = (largura, altura - 64)
        self.rect.x = largura
        
    def update(self):
        if self.escolha == 0:
            self.rect.x -= velocidade_jogo
            if self.rect.topright[0] < 0:
                self.rect.x = largura
            

class DinoVoador (pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.imagens_dinossauro  = []
        for i in range(3,5):
            img = spriteSheet.subsurface((i*32, 0), (32, 32))
            img = pygame.transform.scale(img, (32*3, 32*3))
            self.imagens_dinossauro.append(img)
       
        self.index_lista = 0
        self.image = self.imagens_dinossauro[self.index_lista]
        self.mask = pygame.mask.from_surface(self.image)
        self.escolha = escolha_obstaculo
        self.rect = self.image.get_rect()
        self.rect.center = (largura, 300)
        self.rect.x = largura

        
    def update(self):
        if self.escolha == 1:
            if self.rect.topright[0] < 0:
                self.rect.x = largura
            self.rect.x -= velocidade_jogo
            
            if self.index_lista > 1:
                self.index_lista = 0
            self.index_lista += 0.25
            self.image = self.imagens_dinossauro[int(self.index_lista)]
        

todas_as_sprites = pygame.sprite.Group()
dino = Dino()
todas_as_sprites.add(dino)

for i in range(4):
    nuvem = Nuvens()    
    todas_as_sprites.add(nuvem)

for i in range(11):
    chao = Chao(i)
    todas_as_sprites.add(chao)
    
cacto = Cacto()
todas_as_sprites.add(cacto)

dinoVoador = DinoVoador()
todas_as_sprites.add(dinoVoador)

grupo_obstaculos = pygame.sprite.Group()
grupo_obstaculos.add(cacto)
grupo_obstaculos.add(dinoVoador)

relogio = pygame.time.Clock()

while True:
    relogio.tick(30)
    tela.fill(branco)
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()
        if event.type == KEYDOWN:
            if event.key == K_SPACE and colidiu == False:
                if dino.rect.y != dino.pos_y_inicial:
                    pass
                else:
                    dino.pular()
            if event.key == K_r:
                if colidiu == True:
                    reiniciarJogo()
                    
    colisoes = pygame.sprite.spritecollide(dino, grupo_obstaculos, False, pygame.sprite.collide_mask)
                    
    todas_as_sprites.draw(tela)
    
    if cacto.rect.topright[0] <= 0 or dinoVoador.rect.topright[0] <= 0:
        escolha_obstaculo = choice([0,1])
        cacto.rect.x = largura
        dinoVoador.rect.x = largura
        cacto.escolha = escolha_obstaculo
        dinoVoador.escolha = escolha_obstaculo
    
    if(colisoes and colidiu == False):
        som_colisao.play()
        colidiu = True
    
    if(colidiu == True):
        exibe_mensagem('GAME OVER', 50, preto, largura/2 - 40, altura/2)
        exibe_mensagem(f'Pontuação: {pontos}', 30, preto, largura/2 - 40, altura/2 + 60)
        exibe_mensagem('Aperte R para reiniciar', 30, preto, largura/2 - 40, altura/2 + 90)
    else:
        pontos += 1
        todas_as_sprites.update()
        texto_pontos = exibe_mensagem(pontos, 30, preto, 530, 10)
    
    if pontos % 100 == 0 and pontos != 0:
        som_pontuacao.play()
        if velocidade_jogo >= 23:
            velocidade_jogo += 0
        else:   
            velocidade_jogo += 1
        
        
    pygame.display.flip()

SystemExit: 

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
